# """
    THIS SCRIPT PERFORM PHOSCPREDICTION FROM IMAGE CROPS
    
    1. THE IMAGE CROPS CAN BE FROM A YOLOV3 PREDICTION SCRIPT 
    2. THE IMAGE CROPS CAN BE FROM A ORIGINAL BB FROM LOCALIZATION DATA
    3. THIS SCRIPT PROJECT BACK THE RESULT ON ORIGINAL IMAGE WHERE WE
    CAN SEE CORRECT AND WRONG PREDICTION.
    4. IMAGE CROPS HAS SPECIFIC NAME FORMAT
    
"""

"""
This is PHOSCNET model prediction script
"""

In [27]:
from phoscPipeline import * # contains phosc model 
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label
import os

2022-04-10 06:28:02.168243: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-10 06:28:02.168281: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to m

In [36]:
model=build_model()
MODEL="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/new_IAM_16_"
model.load_weights(MODEL+".h5")
#model=tf.keras.models.load_model(MODEL+".h5")
#print(MODEL)

2022-04-10 06:40:43.185732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-10 06:40:43.185775: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-10 06:40:43.185813: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aniketag-Latitude-7420): /proc/driver/nvidia/version does not exist
2022-04-10 06:40:43.186159: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [38]:
"""
    returns a dictionary having word as a key and PHOSC representation as a value 
"""

def get_comb_label(x):
    phos_labels=gen_label(x)
    phoc_labels=gen_phoc_label(x)
    test_labels=dict()
    for x in phos_labels:
        test_labels[x]=np.concatenate((phos_labels[x],phoc_labels[x]),axis=0)
    return test_labels

#train_word_label=get_comb_label(list(set(df_lex['Word'])))

def similarity(x,y):
    return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


"""
    below predict word from  image crop  image  using PHOSCNET 
"""

def predict(path2Image,test_word_label):
    
    x=img_to_array(load_img(path2Image))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
            
    return op,mx


In [ ]:
testImagesList=os.listdir("./crops/")
testImagesList

In [30]:
"""
    gather all words corresponding to specific single image

"""

import re,os

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"


def getPhoscFromText(testImgPath,orgImage):
    
    """
        input:image directory and image name containing image crops 
        output: dictionary having word as key and PHOSC representation as value
        
        working: input contains paths to dictionary containing all crops of words present in image
        from labels of image in folder, original word labels are extracted and from it PHOSCNET representation 
        is derived
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,imgName in enumerate(os.listdir(testImgPath+orgImage)):

        word=imgName.split("]_")[1]
        word=word.split("_.png")[0]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    #print(len(allWords))
    #allWords
    return test_word_label
    
    
#test_word_label=getPhoscFromText(testImgPath,orgImage)

In [ ]:
#"images=os.listdir("/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//")

In [39]:
"""
    below part takes input single crop folder name and image name as a input
    and perform PHOSC prediction on it, It also plots the prediction on
    original image
"""

import cv2
import os

testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage+"//"
savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual//"
image=cv2.imread(testImgPath+orgImage)
print(" original image name:",orgImage," original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,cropName in enumerate(os.listdir(testCropPath)):
    
    print("")
    print("cropName:",cropName)
    word=cropName.split("]_")[1]
    word=word.split("_.png")[0]
    word= regex.sub('', word)
    
    if len(word)<2: # skip 
        continue
    
    cor=cropName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    predWord,score=predict(testCropPath+cropName)
    
    print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2)
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
    

cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


 original image name: m01-084.png  original image shape: (3542, 2479, 3)

cropName: m01-084.png_25_[1412, 1323, 1612, 1426]_gave_.png
 word: gave   gave  score: 842.1812334847557  coordinate: 1412 1323 1612 1426

cropName: m01-084.png_27_[356, 785, 630, 872]_Deadly_.png


/tmp/ipykernel_913298/4246833656.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


 word: Deadly   deadly  score: 842.3459839726471  coordinate: 356 785 630 872

cropName: m01-084.png_14_[1012, 828, 1026, 863]_,_.png

cropName: m01-084.png_45_[373, 1853, 555, 1911]_voice_.png
 word: voice   voice  score: 901.0204455614288  coordinate: 373 1853 555 1911

cropName: m01-084.png_62_[355, 1679, 665, 1769]_shaving_.png
 word: shaving   shaving  score: 882.8629574675012  coordinate: 355 1679 665 1769

cropName: m01-084.png_55_[1122, 956, 1297, 1046]_early_.png
 word: early   early  score: 854.3177229452145  coordinate: 1122 956 1297 1046

cropName: m01-084.png_6_[538, 1143, 746, 1227]_radio_.png
 word: radio   radio  score: 880.6830746828913  coordinate: 538 1143 746 1227

cropName: m01-084.png_12_[1085, 1690, 1310, 1731]_came_.png
 word: came   came  score: 989.8522366403049  coordinate: 1085 1690 1310 1731

cropName: m01-084.png_63_[1104, 1502, 1192, 1553]_He_.png
 word: He   the  score: 929.392688971249  coordinate: 1104 1502 1192 1553

cropName: m01-084.png_70_[1580, 16

 word: shaving   calling  score: 873.4027262253088  coordinate: 376 1683 656 1767

cropName: m01-084.png_57_[1355, 961, 1511, 1012]_and_.png
 word: and   and  score: 998.5766312971128  coordinate: 1355 961 1511 1012

cropName: m01-084.png_35_[1088, 771, 1321, 857]_deadly_.png
 word: deadly   deadly  score: 826.1069475618345  coordinate: 1088 771 1321 857

cropName: m01-084.png_7_[361, 1325, 484, 1374]_The_.png
 word: The   the  score: 960.7696143358306  coordinate: 361 1325 484 1374
match 47
miss match: 14
 Accuracy: 0.7704918032786885


In [59]:
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"
os.listdir(testCropPath)


['m01-090.png',
 'm01-110.png',
 'old',
 'm01-049.png',
 'm01-060.png',
 'm01-104.png',
 'm01-095.png',
 'm01-121.png',
 'm01-115.png',
 'm01-084.png',
 'm01-079.png']

"""

    below code predicts PHOSC and visualizes all the image present in testCrop folder 
    to add more images into it changes are required in file 
    mnist_testYolo.txt
    
"""

In [60]:
"""
    gather all words
    
"""
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"

images=os.listdir(testCropPath)

savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual//"

print(" images:",images[0] in os.listdir(testImgPath))

"""
for orgImage in images:

    allWords=[]
    #testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"+orgImage
    testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage+"//"

    for indx1,imgName in enumerate(os.listdir(testCropPath)):

        word=imgName.split("]_")[1]
        word=word.split("_.png")[0]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    #len(allWords)
"""
    
for orgImage in images:
    
    matchCount=0
    missMatch=0
    testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"
    
    test_word_label=getPhoscFromText(testCropPath,orgImage)

    testCropPath=testCropPath+orgImage+"//"

    if os.path.isfile(savePath+orgImage+"_singleDict_.png"):
        continue
    
    print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

    image=cv2.imread(testImgPath+orgImage)
    print(" original image shape:",image.shape)

    df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

    
    for indx,cropImgName in enumerate(os.listdir(testCropPath)):

        #print(" cropImgName:",cropImgName)
                    
        #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]

        word=cropImgName.split("]_")[1]
        word=word.split("_.png")[0]

        cor=cropImgName.split("[")[1]
        cor=cor.split("]")[0]
        cor="["+cor+"]"
        cor=eval(cor)

        x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])

        #imgName=imgName.split("_")[0]

        predWord,score=predict(testCropPath+cropImgName,test_word_label)

        print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
        print("")
                
        df.loc[indx,"image_name"]=orgImage
        df.loc[indx,"crop_name"]=cropImgName
        df.loc[indx,"original_word"]=word
        df.loc[indx,"predicted_word"]=predWord        

        if word==predWord:
            #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
            #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            matchCount+=1
        else:

            cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
            cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            missMatch+=1


    print("match",matchCount)
    print("miss match:",missMatch)
    print(" Accuracy:",matchCount/(matchCount+missMatch))

    cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
    cv2.imwrite(savePath+orgImage+"_singleDict_.png",image)
    df.to_csv(savePath+orgImage+"_singleDict_.csv")
    
    #input("check!!!")
    

 images: True
 imageName: m02-102.png  No of unique words: 41  is file: True
 original image shape: (3542, 2479, 3)


/tmp/ipykernel_913298/4246833656.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


 word: at   at  score: 828.0733563253018  coordinate: 1315 2441 1393 2487

 word: managers   managers  score: 894.1417509888499  coordinate: 309 1015 583 1112

 word: bought   bought  score: 903.554680885893  coordinate: 323 1338 538 1460

 word: were   were  score: 972.1222150696993  coordinate: 313 1548 476 1597

 word: centre   centre  score: 986.3987100971801  coordinate: 615 825 795 880

 word: the   the  score: 838.5876016397295  coordinate: 1987 1902 2086 1968

 word: pick   initialled  score: 801.0400749916894  coordinate: 1110 2071 1253 2167

 word: and   and  score: 994.737916000242  coordinate: 311 1179 462 1240

 word: with   with  score: 995.7895401185027  coordinate: 1590 997 1775 1058

 word: of   of  score: 755.4126409889325  coordinate: 1951 2420 2034 2532

 word: initialled   in  score: 919.649212161147  coordinate: 840 2439 908 2492

 word: men   men  score: 851.3405597392955  coordinate: 1858 2083 2009 2132

 word: their   their  score: 963.1209721801102  coordinate

 word: their   their  score: 844.3868764823301  coordinate: 1905 1532 2075 1605

 word: the   them  score: 854.71209685734  coordinate: 454 823 556 871

 word: would   would  score: 996.8327615967363  coordinate: 806 2077 1019 2145

match 79
miss match: 25
 Accuracy: 0.7596153846153846
 imageName: m01-131.png  No of unique words: 41  is file: True
 original image shape: (3542, 2479, 3)
 word: up   up  score: 985.6943535648409  coordinate: 550 1533 618 1608



/tmp/ipykernel_913298/4246833656.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


 word: and   and  score: 991.8131744387017  coordinate: 1922 833 2036 892

 word: gave   decided  score: 977.1905203189667  coordinate: 365 956 598 1058

 word: turning   turning  score: 937.9168872522924  coordinate: 1852 1228 2058 1350

 word: paced   paced  score: 928.3140867741723  coordinate: 510 1117 676 1199

 word: completely   completely  score: 919.4638077516586  coordinate: 1404 1106 1679 1205

 word: at   at  score: 799.7792096377021  coordinate: 1520 1235 1600 1301

 word: like   like  score: 873.1399982923883  coordinate: 689 1386 781 1448

 word: enter   and  score: 661.0750686191926  coordinate: 1683 774 1708 811

 word: gave   gave  score: 928.9097896917241  coordinate: 366 858 507 968

 word: walked   walked  score: 975.6067177245405  coordinate: 1558 1389 1757 1453

 word: of   and  score: 833.0119619789242  coordinate: 1045 830 1104 944

 word: They   they  score: 869.0948829349796  coordinate: 928 1238 1079 1351

 word: he   he  score: 870.5013444988138  coordinate

/tmp/ipykernel_913298/4246833656.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


 word: was   was  score: 926.0743184207977  coordinate: 329 1677 463 1742

 word: now   took  score: 770.9924381367464  coordinate: 1646 1510 1779 1568

 word: mistake   mistake  score: 920.522984397644  coordinate: 840 1082 1058 1200

 word: evening   middleincomebracket  score: 878.1946294192005  coordinate: 348 1322 573 1471

 word: to   to  score: 876.1870223019602  coordinate: 319 1812 393 1913

 word: never   never  score: 956.6618084396997  coordinate: 337 956 477 1027

 word: All   sublil  score: 795.9269968418722  coordinate: 344 1441 437 1556

 word: he   he  score: 914.1678826148891  coordinate: 824 1260 893 1372

 word: had   had  score: 973.2596855727837  coordinate: 937 1266 1038 1371

 word: had   had  score: 975.606007121444  coordinate: 934 1263 1047 1374

 word: Sometimes   she  score: 712.5448341085112  coordinate: 669 816 689 839

 word: lower   middleincomebracket  score: 848.6678439019405  coordinate: 1068 1791 1232 1927

 word: took   took  score: 845.74060954731

NotADirectoryError: [Errno 20] Not a directory: '/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//m02-083.png.csv'

In [62]:
print(allWords)
len(allWords)
#images

['', 'light', 'loses', 'lifetempo', 'he', 'abroad', 'was', 'at', 'apartment', 'to', 'intersection', 'pedestrian', 'precaution', 'no', 'an', 'in', 'as', 'change', 'its', 'Listening', 'stood', 'garaged', 'other', 'listening', 'the', 'At', 'beat', 'there', 'nor', 'For', 'his', 'vain', 'and', 'night', 'for', 'wait', 'earth', 'superfluous', 'heart', 'path', 'car']


41

"""

    BELOW PART CAN BE USED TO VISUALIZE THE PERFORMANCE OF ORIGINAL CROPS,
    REQUIRED CROPS ARE TAKEN FROM PHOSCNET TRAINING DATA.
    THE CROP NAMES DOES NOT INCLUDE WORDS PRESENT INSIDE IT OR COORDINATE TO HANDLE IT
    CSV data_444_april.csv IS USED WHICH HAS REQUIRED INFORMATION

"""

"""

    1. Select test image_name from data_444*.csv and fetch all records related to that.
    
    2. From fetched records get BB and text of location IE create a new dataframe which has BB and corresponding text
    * there is difference between image name and crop name
    
    3. From PHOSCNET test data separate all crops corresponding to above recods and create a separate folder for it,
    here we have only Image name from 1,2 but not corresponding crop name so we need to derive
    it with string matching.
    
    Get the PHOSC vector space for all word labels present in speecific image
    
    4. Pass those crop to PHOSCNET get prediction and visualize it..
    
"""

In [13]:
import pandas as pd
import os
import cv2
import re

"""
    allPaths
"""

csvFile="/home/aniketag/Documents/phd/yolov5/data/datasets/data_444_april.csv"
phoscTestImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest//"
allPhoscTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/IAM_test/"
PhoscOrgTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"

In [4]:

df1=pd.read_csv(csvFile)
print(df1.shape)

phoscTestImg=os.listdir(phoscTestImgPath)
print(" total test images:",len(phoscTestImg))


(115187, 13)
 total test images: 232


In [7]:
"""
1,2.
    input: image name for which BB,Text records are needed
    output: dataframe with relevant information
    
"""

def fetchRecord(df,imageName):
    
    temp=df[df["image_name"]==imageName]
    print(" No of records:",temp.shape)
    return temp


"""
3.
    crops corresponding to specific image file
    a. get list of all crops used for PHOSCNET testing
    b. filter crops necessary for specific image file
    
"""
from shutil import copy2

def filterCrop(allCropList,imageName):
    
    imgCropList=[]
    
    imageName=imageName.split(".png")[0]
    
    cropPhoscTestData="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/cropPhoscTestData//"
    basePhoscData="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/IAM_test//"
    
    for cropName in allCropList:
        
        if imageName in cropName:
            
            imgCropList.append(cropName)

    print(" TOTAL CROPS CORRESPONDING TO IMAGE "+imageName+" ARE:",len(imgCropList))
    
    
    try:
        os.mkdir(cropPhoscTestData+imageName)
    except Exception as e:
        pass
    
    for filterCropName in imgCropList:
        copy2(basePhoscData+filterCropName,cropPhoscTestData+imageName)
        
        
    print(" transferred crop:",len(cropPhoscTestData+imageName)) 
        
    return imgCropList

#imageName="m01-125.png"
#allCropList=os.listdir(allPhoscTestCrops)
#imgCropList=filterCrop(allCropList,imageName)
#imgCropList

def getPhoscFromDF(df):
    
    """
        input: Dataframe containing all image information like text,coordinate etc.
        output: dictionary having word as key and PHOSC representation as value
        
        working: similar to getPhoscFromText()
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,row in df.iterrows():

        word=row["text"]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    print(len(allWords))
    #allWords
    return test_word_label

imageName="m01-084.png"
temp=fetchRecord(df1,imageName)
#temp.columns
#test_word_label=getPhoscFromDF(temp)
#temp.head(5)

"""
    save crops from original coordinates
    As there is no way to map PHOSCNET test data crops to original image 
    
    below function extracts crops from original image, 
    The crops are saved in following format ImgName.png_IndexNumber_[x1,y1, x2, y2]_textString_.png
    
    The crop coordinate are from original dataset It is not predicted by YOLOV3
    
"""

def PhoscOrgCrops(df,imgName):
    
    image=cv2.imread(phoscTestImgPath+imgName)
    
    print(image.shape)
    
    regex = re.compile('[^a-zA-Z]')
    
    
    try:
        os.mkdir(PhoscOrgTestCrops+imgName)
    
    except Exception as e:
        pass
    
    for indx,row in df.iterrows():
        #print(row)
        word=row["text"]
        word= regex.sub('', word)
        
        if len(word)<2:
            continue

        x1,y1,x2,y2=row["org_x1"],row["org_y1"],row["org_x2"],row["org_y2"]
        
        tempName=imgName+"_"+str(indx)+"_"+str([x1,y1,x2,y2])+"_"+word+".png"
        
        cropImage=image[y1:y2,x1:x2]
        #print(cropImage.shape)

        cv2.imwrite(PhoscOrgTestCrops+imgName+"//"+tempName,cropImage)
        


#PhoscOrgCrops(temp,imageName)



 No of records: (60, 13)


,image_name,class,width,height,org_x1,org_y1,org_x2,org_y2,text,x,y,w,h
31429,m01-084.png,1,2479,3542,967,1823,972,1828,.,0.39,0.52,0.00,0.00
31430,m01-084.png,1,2479,3542,352,780,638,878,Deadly,0.20,0.23,0.12,0.03
31431,m01-084.png,1,2479,3542,712,772,1005,841,stillness,0.35,0.23,0.12,0.02
31432,m01-084.png,1,2479,3542,1018,825,1040,870,",",0.42,0.24,0.01,0.01
31433,m01-084.png,1,2479,3542,1092,775,1351,893,deadly,0.49,0.24,0.10,0.03


In [9]:
imageName

'm01-084.png'

(3542, 2479, 3)


In [51]:
"""
    gather all words
"""
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

#testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"
PhoscOrgTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"

images=os.listdir(phoscTestImgPath)

savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual//"

print(" images:",images[0] in os.listdir(testImgPath))


for imgNumber,orgImage in enumerate(images):
    
    try:
        print("")
        
        if os.path.isfile(savePath+orgImage+"_originalCrops_.png"):
            continue

        matchCount=0
        missMatch=0
        PhoscOrgTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"

        temp=fetchRecord(df1,orgImage) # get relevant records

        PhoscOrgCrops(temp,orgImage) # create crops

        test_word_label=getPhoscFromText(PhoscOrgTestCrops,orgImage) # get attribute space

        PhoscOrgTestCrops=PhoscOrgTestCrops+orgImage+"//"


        #print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

        print(" imageName:",orgImage," No of unique words:",len(test_word_label.keys())," is file:",os.path.isfile(phoscTestImgPath+orgImage))


        image=cv2.imread(phoscTestImgPath+orgImage)
        #print(" original image shape:",image.shape)

        df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

        for indx,cropImgName in enumerate(os.listdir(PhoscOrgTestCrops)):

            #print(" cropImgName:",cropImgName)

            #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]

            word=cropImgName.split("]_")[1]
            word=word.split(".png")[0]

            cor=cropImgName.split("[")[1]
            cor=cor.split("]")[0]
            cor="["+cor+"]"
            cor=eval(cor)

            x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])

            #imgName=imgName.split("_")[0]

            predWord,score=predict(PhoscOrgTestCrops+cropImgName,test_word_label)
            predWord=predWord[:-3]
            #print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
            #print("")

            df.loc[indx,"image_name"]=orgImage
            df.loc[indx,"crop_name"]=cropImgName
            df.loc[indx,"original_word"]=word
            df.loc[indx,"predicted_word"]=predWord        

            if word==predWord:
                #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
                #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                matchCount+=1
            else:

                cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
                cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                missMatch+=1


        print("match",matchCount)
        print("miss match:",missMatch)
        print(" Accuracy:",matchCount/(matchCount+missMatch))

        cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
        cv2.imwrite(savePath+orgImage+"_originalCrops_.png",image)
        df.to_csv(savePath+orgImage+"_originalCrops_.csv")
    
    except Exception as e:
        pass
    
    #input("check!!!")
    

 images: True
















 No of records: (116, 13)
(3542, 2479, 3)
 imageName: m02-083.png  No of unique words: 76  is file: True

 No of records: (96, 13)
(3542, 2479, 3)
 imageName: n02-004.png  No of unique words: 62  is file: True
match 32
miss match: 51
 Accuracy: 0.3855421686746988

 No of records: (70, 13)
(3542, 2479, 3)
 imageName: m04-200.png  No of unique words: 49  is file: True
match 21
miss match: 34
 Accuracy: 0.38181818181818183

 No of records: (73, 13)
(3542, 2479, 3)
 imageName: m04-019.png  No of unique words: 46  is file: True
match 16
miss match: 37
 Accuracy: 0.3018867924528302

 No of records: (69, 13)
(3542, 2479, 3)
 imageName: n02-033.png  No of unique words: 49  is file: True
match 29
miss match: 31
 Accuracy: 0.48333333333333334

 No of records: (71, 13)
(3542, 2479, 3)
 imageName: p03-151.png  No of unique words: 43  is file: True
match 19
miss match: 29
 Accuracy: 0.3958333333333333

 No of records: (82, 13)
(3542, 2479, 3)
 imageName: p03-121.png  N

(3542, 2479, 3)
 imageName: n03-038.png  No of unique words: 46  is file: True
match 17
miss match: 36
 Accuracy: 0.32075471698113206

 No of records: (73, 13)
(3542, 2479, 3)
 imageName: p03-029.png  No of unique words: 54  is file: True
match 16
miss match: 52
 Accuracy: 0.23529411764705882

 No of records: (83, 13)
(3542, 2479, 3)
 imageName: m01-104.png  No of unique words: 56  is file: True
match 20
miss match: 48
 Accuracy: 0.29411764705882354

 No of records: (83, 13)
(3542, 2479, 3)
 imageName: m06-067.png  No of unique words: 49  is file: True
match 33
miss match: 31
 Accuracy: 0.515625

 No of records: (75, 13)
(3542, 2479, 3)
 imageName: m04-138.png  No of unique words: 52  is file: True
match 17
miss match: 42
 Accuracy: 0.288135593220339

 No of records: (74, 13)
(3542, 2479, 3)
 imageName: n04-107.png  No of unique words: 44  is file: True
match 17
miss match: 36
 Accuracy: 0.32075471698113206

 No of records: (72, 13)
(3542, 2479, 3)
 imageName: p06-030.png  No of unique

(3542, 2479, 3)
 imageName: n06-119.png  No of unique words: 41  is file: True
match 21
miss match: 27
 Accuracy: 0.4375

 No of records: (96, 13)
(3542, 2479, 3)
 imageName: m04-081.png  No of unique words: 56  is file: True
match 18
miss match: 51
 Accuracy: 0.2608695652173913

 No of records: (92, 13)
(3542, 2479, 3)
 imageName: p02-144.png  No of unique words: 55  is file: True
match 12
miss match: 69
 Accuracy: 0.14814814814814814

 No of records: (81, 13)
(3542, 2479, 3)
 imageName: m04-222.png  No of unique words: 50  is file: True
match 25
miss match: 30
 Accuracy: 0.45454545454545453

 No of records: (82, 13)
(3542, 2479, 3)
 imageName: m04-007.png  No of unique words: 55  is file: True
match 14
miss match: 51
 Accuracy: 0.2153846153846154

 No of records: (52, 13)
(3542, 2479, 3)
 imageName: n04-039.png  No of unique words: 40  is file: True
match 10
miss match: 34
 Accuracy: 0.22727272727272727

 No of records: (69, 13)
(3542, 2479, 3)
 imageName: p03-004.png  No of unique w

match 18
miss match: 46
 Accuracy: 0.28125

 No of records: (101, 13)
(3542, 2479, 3)
 imageName: m02-048.png  No of unique words: 68  is file: True
match 34
miss match: 57
 Accuracy: 0.37362637362637363

 No of records: (30, 13)
(3542, 2479, 3)
 imageName: p03-189.png  No of unique words: 23  is file: True
match 7
miss match: 16
 Accuracy: 0.30434782608695654

 No of records: (60, 13)
(3542, 2479, 3)
 imageName: m01-084.png  No of unique words: 44  is file: True
match 16
miss match: 32
 Accuracy: 0.3333333333333333

 No of records: (83, 13)
(3542, 2479, 3)
 imageName: p03-158.png  No of unique words: 54  is file: True
match 29
miss match: 34
 Accuracy: 0.4603174603174603

 No of records: (96, 13)
(3542, 2479, 3)
 imageName: m04-164.png  No of unique words: 57  is file: True
match 27
miss match: 36
 Accuracy: 0.42857142857142855

 No of records: (71, 13)
(3542, 2479, 3)
 imageName: p03-009.png  No of unique words: 40  is file: True
match 30
miss match: 21
 Accuracy: 0.5882352941176471


match 14
miss match: 39
 Accuracy: 0.2641509433962264

 No of records: (71, 13)
(3542, 2479, 3)
 imageName: n02-082a.png  No of unique words: 48  is file: True
match 22
miss match: 31
 Accuracy: 0.41509433962264153

 No of records: (73, 13)
(3542, 2479, 3)
 imageName: p06-058.png  No of unique words: 50  is file: True
match 7
miss match: 49
 Accuracy: 0.125

 No of records: (77, 13)
(3542, 2479, 3)
 imageName: m02-109.png  No of unique words: 60  is file: True
match 25
miss match: 46
 Accuracy: 0.352112676056338


In [22]:
len(os.listdir(phoscTestImgPath))

232

In [84]:
df1=pd.read_csv("/home/aniketag/Documents/phd/yolov5/data/datasets/data_444_april.csv")
df1.shape

temp=df1[df1["image_name"]=="m01-084.png"]
temp.shape
temp.head(10)

,image_name,class,width,height,org_x1,org_y1,org_x2,org_y2,text,x,y,w,h
31429,m01-084.png,1,2479,3542,967,1823,972,1828,.,0.39,0.52,0.00,0.00
31430,m01-084.png,1,2479,3542,352,780,638,878,Deadly,0.20,0.23,0.12,0.03
31431,m01-084.png,1,2479,3542,712,772,1005,841,stillness,0.35,0.23,0.12,0.02
31432,m01-084.png,1,2479,3542,1018,825,1040,870,",",0.42,0.24,0.01,0.01
31433,m01-084.png,1,2479,3542,1092,775,1351,893,deadly,0.49,0.24,0.10,0.03
31434,m01-084.png,1,2479,3542,1432,785,1690,869,portent,0.63,0.23,0.10,0.02
31435,m01-084.png,1,2479,3542,1727,753,1743,832,!,0.70,0.22,0.01,0.02
31436,m01-084.png,1,2479,3542,363,966,566,1025,Steve,0.19,0.28,0.08,0.02
31437,m01-084.png,1,2479,3542,645,964,1046,1021,awakened,0.34,0.28,0.16,0.02
31438,m01-084.png,1,2479,3542,1135,961,1304,1057,early,0.49,0.28,0.07,0.03


In [22]:
"""
    gather all words
"""

import re,os
regex = re.compile('[^a-zA-Z]')
allWords=[]

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-084.png"]#,"m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"+orgImage

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"+orgImage

for indx,rows in temp.iterrows():
    
    word=rows["text"]
    word= regex.sub('', word)

    if len(word)>1:
        allWords.append(word)
    
allWords=list(set(allWords))
test_word_label=get_comb_label(allWords)
len(allWords)
#allWords

44

In [ ]:
import cv2
import os


testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage+"//"
savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/final//"
image=cv2.imread(testImgPath+orgImage)
print(" original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,imgName in enumerate(os.listdir(testCropPath)):
    
    print(" imgName:",imgName)
    word=imgName.split("]_")[1]
    word=word.split("_.png")[0]
    
    cor=imgName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    
    predWord,score=predict(testCropPath+imgName)
    
    print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
    print("")
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
        
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


In [52]:
import os
os.getcwd()

'/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3'